```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Unit Testing Your Web Application}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

I live in Belmont,
in the San Francisco Bay Area Peninsula.
I wish to acknowledge it as the
ancestral homeland
of the
Ramaytush Ohlone people.

In [65]:
import json
import pyramid.response
import pyramid.config
import httpx

## Pyramid Example

```{=latex}
\begin{frame}
\frametitle{Crash Course in Pyramid}

Some quick examples!

\end{frame}
```

### Static view

```{=latex}
\begin{frame}[fragile]
\frametitle{Static Value}
```

In [19]:
def empty(request):
    return pyramid.response.Response(
        json.dumps({}).encode("ascii"),
        content_type="application/json",
    )
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [15]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Static View Retrieval}
```

In [20]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

### JSON Renderer

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON View}
```

In [62]:
def jsonv(request):
    return {}
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(jsonv, route_name="json", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [63]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON View Retrieval}
```

In [64]:
request = client.get("/json")
request.json()

{}

```{=latex}
\end{frame}
```

### Using Parameters

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View}
```

In [27]:
def params(request):
    return dict(thing=request.params["thing"])
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [28]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View Retrieval}
```

In [30]:
request = client.get("/params?thing=hello")
request.json()

{'thing': 'hello'}

```{=latex}
\end{frame}
```

### Using Route Matches

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View}
```

In [38]:
def matches(request):
    return dict(name=request.matchdict["name"])
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [39]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View Retrieval}
```

In [40]:
request = client.get("/matches/hello")
request.json()

{'name': 'hello'}

```{=latex}
\end{frame}
```

### Using Request Body

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View}
```

In [41]:
def body(request):
    return dict(field=request.json_body["field"])
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches", renderer="json")
    config.add_route("body", "/body")
    config.add_view(body, route_name="body", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [42]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View Retrieval}
```

In [43]:
request = client.post("/body", json=dict(field="hello"))
request.json()

{'field': 'hello'}

```{=latex}
\end{frame}
```

### Using Settings

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View}
```

In [44]:
def setting(request):
    return dict(field=request.registry.settings["field"])
with pyramid.config.Configurator(settings=dict(field="field_value")) as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches", renderer="json")
    config.add_route("body", "/body")
    config.add_view(body, route_name="body", renderer="json")
    config.add_route("setting", "/setting")
    config.add_view(setting, route_name="setting", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [45]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View Retrieval}
```

In [46]:
request = client.get("/setting")
request.json()

{'field': 'field_value'}

```{=latex}
\end{frame}
```

## Pyramid Concepts

### View

```{=latex}
\begin{frame}
\frametitle{View}

Function \pause

Argument: Request \pause

Return value: Response

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route}

Route \pause

to View \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route Predicates}

Route to view can be conditional:\pause

Request type,\pause

Content type,\pause

Arbitrary predicates

\end{frame}
```

### Route

```{=latex}
\begin{frame}
\frametitle{Route}

Path \pause

to route name \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Route: advanced}

\pause

Path fragment matching \pause

...and more

\end{frame}
```

### Registry

```{=latex}
\begin{frame}
\frametitle{Registry: Application Parameters}

\pause

Settings: ad-hoc dictionary

\pause

Utility: Scalable configuration


\end{frame}
```

In [49]:
import zope.interface
import attrs

```{=latex}
\begin{frame}[fragile]
\frametitle{Interfaces and Components}
```

In [57]:
class IValueGetter(zope.interface.Interface):
    def get_value(self) -> int:
        ...

@zope.interface.implementer(IValueGetter)
@attrs.frozen
class ValueStorer:
    _value: int
    
    def get_value(self):
        return self._value

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Interface-based Registry}
```

In [58]:
config = pyramid.config.Configurator()
config.registry.registerUtility(ValueStorer(value=42))
config.registry.getUtility(IValueGetter).get_value()

42

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Why Interface-based Registry?}

Namespaced\pause

Semantically meaningful \pause

Configuration

\end{frame}
```

### Configurator

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator}

Routes \pause

Views \pause

Registry \pause

\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}
```

In [70]:
def root_stuff(config):
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")    
with pyramid.config.Configurator() as config:
    config.include(root_stuff)
    app = config.make_wsgi_app()

In [71]:
client = httpx.Client(app=app, base_url="https://example.com/")

In [72]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}

Include allows: \pause

\begin{itemize}
* Callable \pause
* Callable dotted name \pause
* Module (will use "includeme") \pause
* Module dotted name 

\end{frame}
```

### Scanning

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Scanning}

TBD

\end{frame}
```

## Python Web Concepts

### WSGI

### `httpx`

## Unit Testing

### Mocking Functions

### Mocking Request

### Calling WSGI

### Using `httpx`

## Summary

### Pyramid: Concepts

### Unit Testing: Why?

### Unit Testing: How?

```{=latex}
\end{document}
```